In [2]:
import pandas as pd
import tensorflow as tf
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import eli5

ModuleNotFoundError: No module named 'eli5'

In [96]:
liar=pd.read_csv("test2.tsv", sep='\t')

liar.columns = ['drop', 'ID', 'label', 'statement', 'subject', 'speaker', 'speaker title', 'state', 'party', 'barely_count', 
'false_count', 'half_count', 'mostly_count', 'fire_count', 'location', 'context']

'''
liar2=pd.read_csv("test2.tsv", sep='\t')
liar2.columns = ['drop', 'ID', 'label', 'statement', 'subject', 'speaker', 'speaker title', 'state', 'party', 'barely_count', 'false_count', 'half_count', 'mostly_count', 'fire_count', 'location', 'context']
x_liar2 = liar2['statement']
'''

def int_decode(label):
	if label == 'true':
		label='true'
	elif label == 'half-true':
		label='true'
	elif label == 'barely-true':
		label='false'
	elif label == 'mostly-true':
		label='true'
	elif label == 'false':
		label='false'
	elif label == 'pants-fire':
		label='false'
	return label

def clean_text(text, remove_stopwords = True):
    output = ""
    text = re.sub(r'[^\w\s]','',text).lower() #lower texts
    text = str(text).replace(r'http[\w:/\.]+','') # removing urls
    text = str(text).replace(r'[^\.\w\s]', '') #remove everything but characters and punctuation
    text = str(text).replace(r'\.\.+','.') #replace multiple periods with a single one
    text = str(text).replace(r'\.',' . ') #replace periods with a single one
    text = str(text).replace(r'\s\s+',' ') #replace multiple white space with a single one
    
    output = output + text
    return str(output.strip()) #[1:-3].replace(" ", " ")

'''
label_new = []
label_old = liar['label']
for i in range(0, len(liar)):
	label_new.append(int_decode(label_old[i]))

label_new2 = []
label_old2 = liar2['label']
for i in range(0, len(liar2)):
	label_new2.append(int_decode(label_old2[i]))
'''
    
liar['statement'] = liar['statement'].apply(clean_text)
liar['label'] = liar['label'].apply(int_encode)
print(liar['label'])
'''
x_train = liar['statement'].apply(clean_text)
x_test = liar2['statement'].apply(clean_text)
y_train = label_new
y_test = label_new2
'''

x_train, x_test, y_train, y_test = train_test_split(liar['statement'], liar['label'], test_size=0.40, random_state=42)
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

0       1
1       1
2       0
3       1
4       0
       ..
1261    0
1262    1
1263    1
1264    1
1265    1
Name: label, Length: 1266, dtype: int64


In [52]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=['true', 'false'])
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:367: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 19
	95percentile : 32
	99percentile : 41


preprocessing test...
language: en
test sequence lengths:
	mean : 18
	95percentile : 32
	99percentile : 40


In [54]:
learner.fit_onecycle(5e-1, 2)



begin training using onecycle policy with max lr of 0.5...
Train for 127 steps, validate for 16 steps
Epoch 1/2
127/127 [==============================] - 781s 6s/step - loss: 59.5965 - accuracy: 0.5507 - val_loss: 0.7034 - val_accuracy: 0.5247
Epoch 2/2
127/127 [==============================] - 777s 6s/step - loss: 0.7476 - accuracy: 0.5626 - val_loss: 0.6925 - val_accuracy: 0.5247


In [55]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor.explain('Jesus Christ is the central figure in Christianity.')

['true', 'false']

In [75]:
print(x_test[1])

we know that tax cuts are remarkably unstimulative when you compare them with different kinds of government spending


In [94]:

import eli5
from eli5.lime import TextExplainer


te = TextExplainer(n_samples=500, random_state=42, position_dependent=True)
te.fit(x_test[505], predictor.predict_proba, )
print("Text tweet:", x_test[505])
print("ELI5 Predicted Class:", y_test[505])
HTML(display((te.show_prediction(target_names=[ 'true','false',]))))


Text tweet: the cbo says that if you raise the minimum wage to 1010 an hour half a million people would lose their jobs
ELI5 Predicted Class: 0


<IPython.core.display.HTML object>

In [1]:
print(x_test, y_test)

NameError: name 'x_test' is not defined

In [93]:
print(x_test[505])

the cbo says that if you raise the minimum wage to 1010 an hour half a million people would lose their jobs
